In [38]:
# Importa as bibliotecas necessárias
import pandas as pd
import datetime
import os
import lxml
import html5lib
from bs4 import BeautifulSoup
import requests

In [154]:
PASTA_SAIDA = '../dados/'
CORONABR = '../dados/corona_brasil.csv'
LINK_MINSAUDE = '../dados/auxiliares/link_minsaude.csv'

In [32]:
# Define o link para download
url = 'https://www.saude.gov.br/noticias/agencia-saude/46605-coronavirus-77-mortes-e-2-915-casos-confirmados'
arquivo = requests.get(url).text

In [136]:
# Lê o arquivo HTML com os casos por estado

soup = BeautifulSoup(arquivo, 'lxml') 

table = soup.find_all('table')[0] 

rows = table.find_all('tr')

df = []

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    df.append([ele for ele in cols if ele])
    
df = pd.DataFrame(df)

# Define nome das colunas
df.columns = df.iloc[0]

# Retira colunas desnecessárias para a conciliação da série histórica
df = df.iloc[2:,1:4]

In [137]:
# Converte o arquivo JavaScript para JSON
# Remove entradas impróprias
df = df[df.CONFIRMADOS.notnull()]
df = df[~df['ÓBITOS'].str.contains('%', na = False)] # Remove valores percentuais

In [138]:
# Adiciona coluna de data e renomeia
agora = str(datetime.date.today())

df['date'] = agora
#df
df.rename(columns={'UF':'uf', 'ÓBITOS':'deaths', 'CONFIRMADOS':'cases'}, inplace=True)
df

,uf,cases,deaths,date
3,AC,24,0,2020-03-26
4,AM,67,1,2020-03-26
5,AP,2,0,2020-03-26
6,PA,13,0,2020-03-26
7,RO,5,0,2020-03-26
8,RR,8,0,2020-03-26
9,TO,7,0,2020-03-26
11,AL,11,0,2020-03-26
12,BA,104,0,2020-03-26
13,CE,235,3,2020-03-26


In [153]:
# Carrega a série histórica
corona = pd.read_csv(CORONABR)

corona.append(df)

,uid,date,time,suspects,refuses,confirmado,deads,local,cases,comments,broadcast,deaths,uf
0,11.0,2020-03-25,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,NaN,0,RO
1,12.0,2020-03-25,NaN,NaN,NaN,NaN,NaN,NaN,23,NaN,NaN,0,AC
2,13.0,2020-03-25,NaN,NaN,NaN,NaN,NaN,NaN,54,NaN,NaN,1,AM
3,14.0,2020-03-25,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN,NaN,0,RR
4,15.0,2020-03-25,NaN,NaN,NaN,NaN,NaN,NaN,7,NaN,NaN,0,PA
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,NaN,2020-03-26,NaN,NaN,NaN,NaN,NaN,NaN,25,NaN,NaN,0,MS
29,NaN,2020-03-26,NaN,NaN,NaN,NaN,NaN,NaN,11,NaN,NaN,0,MT
31,NaN,2020-03-26,NaN,NaN,NaN,NaN,NaN,NaN,102,NaN,NaN,0,PR
32,NaN,2020-03-26,NaN,NaN,NaN,NaN,NaN,NaN,122,NaN,NaN,1,SC


In [156]:
# Exporta a base em CSV
dados = os.path.join(PASTA_SAIDA, 'corona_brasil' + '.csv')
corona.append(df).to_csv(dados, index = False)